In [1]:
import tensorflow as tf
import numpy as np
import keras_tuner as kt
import pandas as pd
import seq_generator
import tensorflow.keras.layers as layers
import tensorflow_datasets as tfds
import json
import os

pars = json.load(open("tuner_run/best_hps.json"))

train_ds , val_ds = seq_generator.get_train_and_val(batch_size=128)
train_ds = train_ds.map(lambda x,y: (x,tf.one_hot(y,2)))
val_ds = val_ds.map(lambda x,y: (x,tf.one_hot(y,2)))


def build_cnn():

    inputs = tf.keras.Input([260,260,3])

    img_augmentation = tf.keras.models.Sequential(
        [
            tf.keras.layers.RandomRotation(factor=0.15),
            tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
            tf.keras.layers.RandomContrast(factor=0.1),
        ],
        name="img_augmentation",
    )

    x = tf.keras.layers.Resizing(224,224) (inputs)

    x = img_augmentation(x)

    base = tf.keras.applications.EfficientNetB0(
                include_top=False,
                weights='imagenet',
                input_tensor= x)

    base.trainable = False

    x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(base.output)

    return tf.keras.Model(inputs,x)

def build_model(hps,cnn):


    inputs = tf.keras.Input([5,260,260,3])
    x = layers.TimeDistributed(cnn)(inputs)

    drop1 = hps["drop1"]

    x = layers.TimeDistributed( layers.Dropout(drop1), name = "drop1") (x)

    hidden_units =  hps["hidden_units"]

    if hps["rnn"] == "lstm":
        x = layers.LSTM(hidden_units, name = "lstm")(x)
    else:
        x = layers.GRU(hidden_units, name = "gru") (x)

    drop2 = hps["drop2"]
    x = layers.Dropout(drop2,name ="drop2")(x)

    outputs = tf.keras.layers.Dense(2, activation="softmax", name="pred",kernel_regularizer=tf.keras.regularizers.L2())(x)

    learning_rate = hps["lr"]

    model = tf.keras.Model(inputs,outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    return model

def warm_model(cnn):
    cnn.trainable = True
    for layer in cnn.layers:
            if isinstance(layer, tf.keras.layers.BatchNormalization):
                layer.trainable = False

    for layer in cnn.layers[:-20]:

            layer.trainable = False

    return cnn


def get_callbacks(name):
    log_dir = f"tuner_run/ensemble/lots/{name}/"
    save_dir = f"tuner_run/ensemble/weights/{name}_warm/"
    os.makedirs(save_dir,exist_ok=True)
    os.makedirs(log_dir,exist_ok=True)
    callbacks = []
    callbacks.append(tf.keras.callbacks.ModelCheckpoint(
    monitor='val_accuracy',
    filepath=save_dir + "cp.cpkt",
    verbose=1,
    save_weights_only=True,
    save_best_only=True))

    callbacks.append(tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    verbose=1,
    mode='auto'))

    callbacks.append( tf.keras.callbacks.TensorBoard(log_dir=log_dir))

    return callbacks


def run(name):

    cnn = build_cnn()
    model = build_model(pars,cnn)
    print("jo")
    model.fit(train_ds, epochs=1, validation_data=val_ds)
    print("was")
    save_dir = f"tuner_run/ensemble/weights/{name}_cold/"
    os.makedirs(save_dir,exist_ok=True)
    model.save_weights(save_dir + "cp.cpkt")


    cnn = warm_model(cnn)
    model = build_model(pars,cnn)
    model.load_weights(save_dir + "cp.cpkt")

    callbacks = get_callbacks(name)
    model.fit(train_ds, epochs=1, validation_data=val_ds,callbacks=callbacks)



c:\Users\ual\.conda\envs\felix\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
perfomances = {}
for i in range(10):

    tf.keras.backend.clear_session()
    name = f"b0_ensemple_{i}"
    cnn = build_cnn()

    model = build_model(pars,cnn)

    model
    print("jo")
    model.fit(train_ds, epochs=1, validation_data=val_ds)

    print("was")
    save_dir = f"tuner_run/ensemble/weights/{name}_cold/"
    os.makedirs(save_dir,exist_ok=True)
    model.save_weights(save_dir + "cp.cpkt")


    cnn = warm_model(cnn)
    model = build_model(pars,cnn)
    model.load_weights(save_dir + "cp.cpkt")

    callbacks = get_callbacks(name)
    hist = model.fit(train_ds, epochs=7, validation_data=val_ds,callbacks=callbacks)
    val_acc = hist.history['val_accuracy']
    max_acc = max(val_acc)

    perfomances[i] = val_acc

    print(i,val_acc)

jo
283/283 [==============================] - 126s 403ms/step - loss: 0.1439 - accuracy: 0.9579 - val_loss: 0.4479 - val_accuracy: 0.8277
was
Epoch 1/7
283/283 [==============================] - ETA: 0s - loss: 0.0677 - accuracy: 0.9832
Epoch 1: val_accuracy improved from -inf to 0.92241, saving model to tuner_run/ensemble/weights/b0_ensemple_0_warm\cp.cpkt
283/283 [==============================] - 133s 438ms/step - loss: 0.0677 - accuracy: 0.9832 - val_loss: 0.2242 - val_accuracy: 0.9224
Epoch 2/7
283/283 [==============================] - ETA: 0s - loss: 0.0275 - accuracy: 0.9951
Epoch 2: val_accuracy did not improve from 0.92241
283/283 [==============================] - 121s 428ms/step - loss: 0.0275 - accuracy: 0.9951 - val_loss: 0.2789 - val_accuracy: 0.9000
Epoch 3/7
283/283 [==============================] - ETA: 0s - loss: 0.0200 - accuracy: 0.9966
Epoch 3: val_accuracy did not improve from 0.92241
283/283 [==============================] - 121s 427ms/step - loss: 0.0200 - ac

In [3]:
for i in range(10,20):

    tf.keras.backend.clear_session()
    name = f"b0_ensemple_{i}"
    cnn = build_cnn()

    model = build_model(pars,cnn)

    model
    print("jo")
    model.fit(train_ds, epochs=1, validation_data=val_ds)

    print("was")
    save_dir = f"tuner_run/ensemble/weights/{name}_cold/"
    os.makedirs(save_dir,exist_ok=True)
    model.save_weights(save_dir + "cp.cpkt")


    cnn = warm_model(cnn)
    model = build_model(pars,cnn)
    model.load_weights(save_dir + "cp.cpkt")

    callbacks = get_callbacks(name)
    hist = model.fit(train_ds, epochs=7, validation_data=val_ds,callbacks=callbacks)
    val_acc = hist.history['val_accuracy']
    max_acc = max(val_acc)

    perfomances[i] = val_acc

    print(i,val_acc)

jo
283/283 [==============================] - 120s 401ms/step - loss: 0.1441 - accuracy: 0.9585 - val_loss: 0.2847 - val_accuracy: 0.8962
was
Epoch 1/7
283/283 [==============================] - ETA: 0s - loss: 0.0874 - accuracy: 0.9763
Epoch 1: val_accuracy improved from -inf to 0.89551, saving model to tuner_run/ensemble/weights/b0_ensemple_10_warm\cp.cpkt
283/283 [==============================] - 132s 435ms/step - loss: 0.0874 - accuracy: 0.9763 - val_loss: 0.2845 - val_accuracy: 0.8955
Epoch 2/7
283/283 [==============================] - ETA: 0s - loss: 0.0302 - accuracy: 0.9949
Epoch 2: val_accuracy improved from 0.89551 to 0.93464, saving model to tuner_run/ensemble/weights/b0_ensemple_10_warm\cp.cpkt
283/283 [==============================] - 121s 425ms/step - loss: 0.0302 - accuracy: 0.9949 - val_loss: 0.1921 - val_accuracy: 0.9346
Epoch 3/7
283/283 [==============================] - ETA: 0s - loss: 0.0237 - accuracy: 0.9956
Epoch 3: val_accuracy did not improve from 0.93464
2

In [10]:
perfomances = {i: max(perfomances[i]) for i in perfomances }

In [11]:
with open("tuner_run/ensemble_results.json",'w') as f:

    json.dump(perfomances,f)

In [20]:
val = np.array(list(perfomances.values()))
np.argsort(-val)[:5]

array([ 4,  2,  1, 16, 17], dtype=int64)

In [19]:
val[awaitnp.argsort(-val)]

array([0.94886619, 0.94219655, 0.9417519 , 0.94130725, 0.9377501 ,
       0.93641621, 0.93574923, 0.93485993, 0.93463761, 0.93241441,
       0.93108046, 0.92930192, 0.92796797, 0.92752331, 0.9237439 ,
       0.9237439 , 0.9228546 , 0.92240995, 0.92152065, 0.91974211])

In [25]:
def load_model(i):
    cnn = build_cnn()

    model = build_model(pars,cnn)
    name = f"b0_ensemple_{i}"
    model.load_weights(f"tuner_run/ensemble/weights/{name}_warm/cp.cpkt")
    return model

_ , val_ds = seq_generator.get_train_and_val(batch_size=20)
val_ds = val_ds.map(lambda x,y: (x,tf.one_hot(y,2)))


inputs = tf.keras.Input([5,260,260,3])
x = [load_model(i)(inputs) for i in [ 4,  2,  1, 16, 17]]

outputs = tf.math.reduce_mean(x,axis = 0)

ensemble = tf.keras.Model(inputs,outputs)
ensemble.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
ensemble.evaluate(val_ds)

225/225 [==============================] - 88s 318ms/step - loss: 0.1913 - accuracy: 0.9515


[0.19134901463985443, 0.9515340328216553]

In [26]:
inputs = tf.keras.Input([5,260,260,3])
x = [load_model(i)(inputs) for i in [ 4,  2,  1]]

outputs = tf.math.reduce_mean(x,axis = 0)

ensemble = tf.keras.Model(inputs,outputs)
ensemble.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
ensemble.evaluate(val_ds)

225/225 [==============================] - 54s 194ms/step - loss: 0.1977 - accuracy: 0.9518


[0.19770410656929016, 0.9517563581466675]

In [56]:
tf.keras.backend.clear_session()

val_df = pd.read_feather('new_train_seqs.df').set_index('index').query('ds_type == "val"')
val_df['labels'] = (val_df.view_direction == 'Sideways').astype(float)


bce = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

def get_errors(i):
    tf.keras.backend.clear_session()
    df = val_df.copy()
    model = load_model(i)
    preds = model.predict(val_ds,verbose= 1)[:,1]
    df['loss'] = bce(df.labels.values[:,None],preds[:,None])

    return df[(0.5 < df.loss)].index, df[(0.5 < preds) != df.labels].index 

errors = [get_errors(i) for i in [ 4,  2,  1, 16, 17]]

225/225 [==============================] - 17s 67ms/step


In [64]:
low_loss  = np.concatenate([i[0] for i in errors])
fp  = np.concatenate([i[1] for i in errors])
inds,cs = np.unique(low_loss,return_counts=1)
fp,fcs = np.unique(low_loss,return_counts=1)

In [68]:
import tqdm, matplotlib.pyplot as plt

# val_df = pd.read_feather('imgl.df').set_index('index')
func = lambda s: s[s.find("images/")+7:]
val_df["local_path"] = np.array([[f"../seq_data/{s}/{func(i)}"for i in c] for c,s in zip(val_df.Paths,val_df.ds_type)]).tolist()
high_loss = val_df.loc[inds[cs> 1]]

p = "tmp/ensemble_errors/"
os.makedirs(p,exist_ok=True)

for n in tqdm.tqdm(range(len(high_loss))):



    row = high_loss.iloc[n]
    fig, ax = plt.subplots(2,2,figsize=(12,12))
    keys = row['local_path']
    #np.random.shuffle(keys)
    for i,a in enumerate(ax.flatten()):
        try:
            a.imshow(plt.imread(keys[i]))
        except: pass
        a.axis('off')

    label = row['view_direction']
    if high_loss.index[n] in fp:
    
        pre = fcs[fp ==high_loss.index[n]] 

    else:

        pre = 0

    text = f"Label: {row['view_direction']}, N False Pred: {pre[0]}, N High Loss: {cs[inds ==high_loss.index[n]][0]}"

    plt.suptitle(text)
    path = p +  str(high_loss.index[n]) + ".jpg"
    plt.savefig(path)
    plt.close()

100%|██████████| 345/345 [00:47<00:00,  7.32it/s]


In [80]:
df = pd.read_feather('cleaned_seqs.df').set_index('index')

In [81]:
front = [int(i[:-4]) for i in os.listdir('tmp/front')]
side = [int(i[:-4]) for i in os.listdir('tmp/side')]

change =( df.loc[front, "view_direction"] != "Forward").sum() + ( df.loc[side, "view_direction"] != "Sideways").sum()
change/len(df)*0.1

0.00027476179924662087

In [83]:
df.loc[front, "view_direction"] = "Forward"
df.loc[side, "view_direction"] = "Sideways"

df.reset_index().to_feather('cleaned_seqs.df')

In [82]:
losses = []
for i in range(20):


In [ ]:
a = model.evaluate(val_ds.take(2))

: 

: 